In [0]:
!pip install databricks-sdk -U
dbutils.library.restartPython()

In [0]:
%sql

CREATE WIDGET TEXT CATALOG_NAME DEFAULT "pedroz_catalog";
CREATE WIDGET TEXT SCHEMA_NAME DEFAULT "metric_views_schema";

-- This is just a simple query example, demonstrating that you can query Metric Views just like any other view!
SELECT
  dayOfWeek,
  MEASURE(`sales_sum`),
  MEASURE(`sales_avg`),
  MEASURE(`sales_stddev`),
  MEASURE(`sales_quartile1`),
  MEASURE(`sales_quartile2`),
  MEASURE(`sales_quartile3`),
  MEASURE(`sales_quartile4`),
  MEASURE(`sales_largest`),
  MEASURE(`sales_smallest`)
FROM ${CATALOG_NAME}.${SCHEMA_NAME}.sales_relationships
GROUP BY ALL
ORDER BY dayOfWeek

You can also consume Metric Views in Genie:
<img src="./figures/metric_view_in_genie.png" width="75%" />

Check out the sample serialized Genie space [./dashboard_and_genie/genie_space.json](./dashboard_and_genie/dashboard.json). 
You can use the code snippet below to automatically create it in your workspace, it uses the [Create Genie Space](https://docs.databricks.com/api/workspace/genie/createspace) API.  

In [0]:
import json
from databricks.sdk import WorkspaceClient

# Load the serialized Genie space from the JSON file
with open('./dashboard_and_genie/genie_space.json', 'r') as f:
    genie_space_json_str = f.read()

# Replace CATALOG_NAME and SCHEMA_NAME placeholders with parameter values
catalog_name = dbutils.widgets.get("CATALOG_NAME")
schema_name = dbutils.widgets.get("SCHEMA_NAME")
genie_space_json_str = genie_space_json_str.replace("CATALOG_NAME", catalog_name).replace("SCHEMA_NAME", schema_name)

# Specify the warehouse ID (replace with your actual warehouse ID)
dbutils.widgets.text("warehouse_id", "8baced1ff014912d")
warehouse_id = dbutils.widgets.get("warehouse_id")

# Prepare the payload as required
payload = {
    "serialized_space": genie_space_json_str,
    "warehouse_id": warehouse_id,
    "title": "Sample Sales Genie Space (created via API)"
}

# Initialize WorkspaceClient (will use DATABRICKS_HOST and DATABRICKS_TOKEN from env)
w = WorkspaceClient()
workspace_url = w.config.host
pat_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

import requests

headers = {
    "Authorization": f"Bearer {pat_token}",
    "Content-Type": "application/json"
}

# POST to Genie Create Space API
response = requests.post(
    f"{workspace_url}/api/2.0/genie/spaces",
    headers=headers,
    data=json.dumps(payload)
)

response.raise_for_status()
space_info = response.json()
print(f"Created Genie space with ID *{space_info['space_id']}* and name *{space_info['title']}*")

And create dashboards using it!  
<img src="./figures/metric_view_in_dashboards.png" width="75%" />

Check out the [./dashboard_and_genie/dashboard.lvdash.json](./dashboard_and_genie/dashboard.lvdash.json) dashboard for a simple dashboard that can be used as an example to show how you can consume Metric Views using AIBI Dashboards.   

**Note**: You will need to replace the catalog and schema names from the dashboard.lvdash.json file from **pedroz_catalog** and **metric_views_schema** to your desired catalog and schema names. The script below does that automatically, and creates another .lvdash.json file which you can edit called [./dashboard_and_genie/dashboard_parameterized.lvdash.json](./dashboard_and_genie/dashboard_parameterized.lvdash.json)

In [0]:
catalog_name = dbutils.widgets.get("CATALOG_NAME")
schema_name = dbutils.widgets.get("SCHEMA_NAME")

with open('./dashboard_and_genie/dashboard.lvdash.json', 'r') as f:
    dashboard_json_str = f.read()

dashboard_json_str = dashboard_json_str.replace("CATALOG_NAME", catalog_name).replace("SCHEMA_NAME", schema_name)

with open('./dashboard_and_genie/dashboard_parameterized.lvdash.json', 'w') as f:
    f.write(dashboard_json_str)